# Lat Long Comparer
#### This notebook computes the geometry of the bus stops vs their lat long and sorts by their difference. 

In [22]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

token = config['DEFAULT']['READ_KEY']

In [29]:
import math
import pandas as pd
from pprint import pprint
from amigocloud import AmigoCloud

ac = AmigoCloud(token)
me = ac.get('me')
me['id']

PROJECT_OWNER = me['id']
PROJECT_ID = 3098
DATASET_ID = 27816

dataset_url = '/users/{user_id}/projects/{project_id}/datasets/{dataset_id}'
dataset = ac.get(dataset_url.format(user_id=PROJECT_OWNER,
                                    project_id=PROJECT_ID,
                                    dataset_id=DATASET_ID))
table_name = dataset['table_name']

print('The table name we have to use in SQL queries is', table_name)
print('The table has', dataset['feature_count'], 'rows')

sql_url = '/users/{user_id}/projects/{project_id}/sql'.format(
    user_id=PROJECT_OWNER, project_id=PROJECT_ID
)
#,ST_X(geom) as lat, ST_Y(geom) as lon
query = 'SELECT *,ST_X(geom), ST_Y(geom) FROM {table}'.format(table=table_name)
total_rows = dataset['feature_count']
offset = 0
limit = 1000
rows = []
response = ac.get(dataset['master'])
master = response['master']

#while len(rows) < total_rows:
while len(rows) < total_rows:
    response = ac.get(sql_url, {'query': query, 'offset': offset,
                                'limit': limit, 'state': master,
                                'dataset_id': DATASET_ID})

    if not offset:  # i.e. If first request
        print('The schema of the result is:')
        pprint(response['columns'])

    fetched_rows = len(response['data'])
    offset += fetched_rows
    rows += response['data']
    # print 'Fetched', fetched_rows, 'rows'

# print 'These are all the rows:'
#pprint(rows)

# response = ac.get(sql_url, {'query': query, 'offset': offset,'limit': limit, 
#                             'state': master,'dataset_id': DATASET_ID})
df = pd.DataFrame()
inv_update= df.from_dict(rows,orient='columns')
inv_update

The table name we have to use in SQL queries is dataset_27816
The table has 3985 rows
The schema of the result is:
[{'name': 'ogc_fid', 'type': 'integer'},
 {'name': 'lat', 'type': 'string'},
 {'name': 'long', 'type': 'string'},
 {'name': 'feature', 'type': 'string'},
 {'name': 'rtiid', 'type': 'string'},
 {'name': 'stop', 'type': 'string'},
 {'name': 'route_number', 'type': 'string'},
 {'name': 'routes_listed', 'type': 'string'},
 {'name': 'rti', 'type': 'string'},
 {'name': 'standard_vta_sign', 'type': 'string'},
 {'name': 'two_sided_sign', 'type': 'string'},
 {'name': 'other_agencies_signs', 'type': 'string'},
 {'name': 'rti_decal', 'type': 'string'},
 {'name': 'others_using_stop', 'type': 'string'},
 {'name': 'adopt_a_stop', 'type': 'string'},
 {'name': 'information_sign', 'type': 'string'},
 {'name': 'other_info_sign', 'type': 'string'},
 {'name': 'tactile_signs', 'type': 'string'},
 {'name': 'pole', 'type': 'string'},
 {'name': 'other_pole', 'type': 'string'},
 {'name': 'vta_benc

,ac_asphalt_paving,accessible_path_to_crossing,ad_bench,ada_accessible,adjacent_property,adopt_a_stop,amigo_id,atname,bench_pad,boarding_area,...,stopname,street_view,tactile_signs,tc_other,tc_with_shelter,trapeze_id,trapeze_status,two_sided_sign,vta__trash_container,vta_bench
0,Not Applicable,Yes,0,Yes,Commercial,No,f66d7a6622fe40ecac9b04d827b5a19c,MERIDIAN,No,Yes,...,SAN CARLOS & MERIDIAN,https://maps.google.com/maps?q=&layer=c&cbll=3...,None,None,Not Applicable,6001.0,Active,No,0,1
1,None,None,None,None,None,None,a5063e592ac44b61a06519e31e19b806,None,None,None,...,None,None,None,None,None,NaN,None,None,None,None
2,Not Applicable,Yes,0,Yes,School,No,7188f5bb8e2c47158634e9f8abcdd677,STEVENS CREEK,No,Yes,...,STELLING & STEVENS CREEK (FS),https://maps.google.com/maps?q=&layer=c&cbll=3...,None,City Owned,No,6008.0,Active,No,1,0
3,Not Applicable,Yes,0,Yes,Vacant Lot,No,165fcc4ec05844beb9a4ef98f9c0c9d4,SIERRA - FLEA MARKET,No,Yes,...,BERRYESA & SIERRA - FLEA MARKET,https://maps.google.com/maps?q=&layer=c&cbll=3...,Bus,None,None,72.0,None,None,0,0
4,Back of Curb,,0,None,None,Yes,3bc109b3027b45798397c6ea47cd62dd,TRINIDAD,Yes,,...,ALMADEN & TRINIDAD,https://maps.google.com/maps?q=&layer=c&cbll=3...,None,None,None,122.0,None,,0,1
5,Not Applicable,Yes,0,Yes,Residential,Yes,c73b0e0cd40f4737bd457095b5bdaf54,4TH,Yes,Yes,...,HEDDING & 4TH,https://maps.google.com/maps?q=&layer=c&cbll=3...,Bus,None,No,78.0,None,None,1,1
6,Not Applicable,,0,None,None,No,83213a45066a43798951e05d4b5f1b8d,ALMADEN,No,,...,VIA VALIENTE & ALMADEN,https://maps.google.com/maps?q=&layer=c&cbll=3...,Bus,None,None,123.0,None,,0,1
7,Not Applicable,,0,None,None,No,3130425750dc4207b54557b9c0367aed,COCHRANE CIRCLE,No,,...,COCHRANE & COCHRANE CIRCLE,https://maps.google.com/maps?q=&layer=c&cbll=3...,None,None,None,217.0,None,,0,0
8,Not Applicable,,0,None,None,No,d97bc9032ad9482d97df49248ece832d,MOBILE HOME ESTATES,No,,...,MOBILE HOME ESTATES,https://maps.google.com/maps?q=&layer=c&cbll=3...,None,None,None,221.0,None,,0,1
9,Not Applicable,,0,None,None,No,3021badf059c4fca8208135cca86deea,COCHRANE CIRCLE,No,,...,COCHRANE & COCHRANE CIRCLE,https://maps.google.com/maps?q=&layer=c&cbll=3...,Bus,None,None,222.0,None,,0,0


In [30]:
inv_update[['lat','long','st_y','st_x']].head()

,lat,long,st_y,st_x
0,37.323627,-121.913391,37.323619,-121.913369
1,None,None,37.293955,-121.872703
2,37.322226,-122.041663,37.322225,-122.041631
3,37.370016,-121.878973,37.370016,-121.878973
4,37.21525843,-121.8536351,37.215258,-121.853635


In [31]:
inv_update['long_f'] = inv_update['long'].str.strip().astype(float)
inv_update['lat_f'] = inv_update['lat'].str.strip().astype(float)
#inv_update['st_y']
# inv_update.loc[inv_update['lat']!='',]['lat']

In [32]:
inv_update['long_diff'] = (inv_update['long_f'] - inv_update['st_x']).abs()
inv_update['lat_diff'] = (inv_update['lat_f'] - inv_update['st_y']).abs()

In [33]:
inv_update['diff'] = inv_update['long_diff'] + inv_update['lat_diff']

In [44]:
inv_update.to_csv('lat_long_compare.csv')

In [34]:
inv_update.sort_values(by='diff',ascending=False)[['stop','diff','lat_diff','long_diff','rtiid']]

,stop,diff,lat_diff,long_diff,rtiid
2819,327910,0.010001,0.003798,0.006203,65061
685,297601,0.003859,0.003098,0.000762,62524
82,338002,0.003074,0.000013,0.003060,61372
2011,338004,0.003011,0.000001,0.003009,65702
600,307006,0.002180,0.000018,0.002163,60380
3571,546201,0.001927,0.000344,0.001583,61251
228,277204,0.001426,0.000082,0.001345,61331
2627,267901,0.001189,0.000346,0.000842,64468
3387,327304,0.001183,0.000936,0.000247,62422
767,227001,0.001024,0.001010,0.000014,62394
